## Project Overwiev of the data: measuremnts 

## 1. Overwiev of the ncfile of  choice 

In [3]:
#Import all needed libraries: 
import os
import numpy as np
import pandas as pd
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [11]:
bouy6_path = 'data/measurements/2023-11-06_Buoy6_BSH_N-9.nc'
bouy2_path = 'data/measurements/2023-11-09_Buoy2_BSH_N-9.nc'
ncfile = Dataset(bouy2_path)

In [12]:
# Print all the head groups first with enumeration
for group_count, group_name in enumerate(ncfile.groups):
    group = ncfile.groups[group_name]
    print(f"Top Group Nr. {group_count + 1}: {group_name}")


Top Group Nr. 1: ZX_LIDAR_WLBZ_2
Top Group Nr. 2: METEO_WLBZ_2
Top Group Nr. 3: ADD_DATA_WLBZ_2
Top Group Nr. 4: ZX_LIDAR_WLBZ_6_MCP
Top Group Nr. 5: METEO_WLBZ_6_MCP


In [13]:
# Iterate over all the groups
for group_name in ncfile.groups:
    group = ncfile.groups[group_name]
    print(f"Group: {group_name}")

    # Iterate over all variables of the group 
    for variable_group in group.variables:
        group_variable = group.variables[variable_group]
        print(f"    Groupvariable: {variable_group}")
    
    # Iterate over all the subgroups
    for subgroup_name in group.groups:
        subgroup = group.groups[subgroup_name]
        print(f"        Subgroup: {subgroup_name}")
        
        # Iterate over all the variables in the subgroup
        for variable_name in subgroup.variables:
            variable = subgroup.variables[variable_name]
            print(f"            SubGroupVariable: {variable_name}")



Group: ZX_LIDAR_WLBZ_2
    Groupvariable: wind_speed
    Groupvariable: wind_from_direction
    Groupvariable: wind_from_direction_availability
    Groupvariable: wind_speed_availability
    Groupvariable: wind_speed_min
    Groupvariable: wind_speed_max
    Groupvariable: wind_speed_stddev
    Groupvariable: wind_count
    Groupvariable: turbulence_intensity
    Groupvariable: algorithm_bitmask
Group: METEO_WLBZ_2
        Subgroup: Vaisala Weather Station
            SubGroupVariable: rain_accumulation
            SubGroupVariable: rain_accumulation_calculated
            SubGroupVariable: rain_duration
            SubGroupVariable: rain_intensity
            SubGroupVariable: hail_accumulation
            SubGroupVariable: hail_duration
            SubGroupVariable: hail_intensity
            SubGroupVariable: precipitation_count
            SubGroupVariable: air_temperature
            SubGroupVariable: relative_humidity
            SubGroupVariable: air_pressure
            SubGrou

In [14]:
# Iterate over the variables in the certain group in one of the nc files 
group_name = 'ZX_LIDAR_WLBZ_2'

for variable_name in ncfile.groups[group_name].variables:
    variable = ncfile.groups[group_name].variables[variable_name]
    print(f"Variable Name: {variable_name}")
    print(f"Variable Attributes:")
    print(f"    Units: {variable.units}")
    print(f"    Long Name: {variable.long_name}")
    print(f"    Shape: {variable.shape}")
    print(f"---------------------------------------------------------")


Variable Name: wind_speed
Variable Attributes:
    Units: m/s
    Long Name: Wind speed
    Shape: (67680, 1, 1, 6)
---------------------------------------------------------
Variable Name: wind_from_direction
Variable Attributes:
    Units: degree
    Long Name: Horizontal wind direction
    Shape: (67680, 1, 1, 6)
---------------------------------------------------------
Variable Name: wind_from_direction_availability
Variable Attributes:
    Units: percent
    Long Name: Availability of the horizontal wind direction
    Shape: (67680, 1, 1, 6)
---------------------------------------------------------
Variable Name: wind_speed_availability
Variable Attributes:
    Units: percent
    Long Name: Availability of the horizontal wind speed
    Shape: (67680, 1, 1, 6)
---------------------------------------------------------
Variable Name: wind_speed_min
Variable Attributes:
    Units: m/s
    Long Name: Minimum wind speed
    Shape: (67680, 1, 1, 6)
----------------------------------------

In [15]:
# Iterate over the subgroups in  a certain group
subgroup_parent = ncfile.groups['METEO_WLBZ_2'] #fill in the group of choice here! 

for subgroup_name in subgroup_parent.groups:
    subgroup = subgroup_parent.groups[subgroup_name]
    print(f"SUBGROUP: {subgroup_name}")
    
    # Iterate over the variables in the subgroup and print their attributes
    for variable_name in subgroup.variables:
        variable = subgroup.variables[variable_name]
        print(f"Variable Name: {variable_name}")
        print(f"Variable Attributes:")
        print(f"    Units: {variable.units}")
        print(f"    Long Name: {variable.long_name}")
        print(f"    Shape: {variable.shape}")
        print(f"---------------------------------------------------------")



SUBGROUP: Vaisala Weather Station
Variable Name: rain_accumulation
Variable Attributes:
    Units: mm
    Long Name: Rain accumulation measured by the Vaisala Weather Station at 3 m above MSL
    Shape: (67680, 1, 1)
---------------------------------------------------------
Variable Name: rain_accumulation_calculated
Variable Attributes:
    Units: mm
    Long Name: Rain accumulation calculated from rain duration and rain intensity data provided by the Vaisala Weather Station at 3 m above MSL
    Shape: (67680, 1, 1)
---------------------------------------------------------
Variable Name: rain_duration
Variable Attributes:
    Units: s
    Long Name: Rain duration measured by the Vaisala Weather Station at 3 m above MSL
    Shape: (67680, 1, 1)
---------------------------------------------------------
Variable Name: rain_intensity
Variable Attributes:
    Units: mm/h
    Long Name: Rain intensity measured by the Vaisala Weather Station at 3 m above MSL
    Shape: (67680, 1, 1)
--------